#### An experimentation with multiple choice questions created using [stanza's](https://stanfordnlp.github.io/stanza/) Ancient [Greek](https://stanfordnlp.github.io/stanza/available_models.html) parser and [jtauber's](https://github.com/jtauber) accentuation rules

---
## Resources
greek-accentuation [documentation](https://github.com/jtauber/greek-accentuation/blob/master/docs.rst)

greek-normalization [documentation](https://github.com/jtauber/greek-normalisation/blob/master/tests.rst)

---
#### Known issues: 
- Sometimes these exercises are finicky and require restarting the kernel and re-importing greek_accentuation.accentuation
- In exercise 4, if a word contains more than one syllable that is spelled the same (as with φιλοσοφια) and the user selects either one of them, Gradio automatically selects the other as well

---
#### Notes/ Future Updates: 
- The feedback that Exercise 2 gives is fairly limited because I'm generating the impossible accentuations randomly, rather than according to any rules. Exercise 5 is an attempt to remedy this issue

---
---

## Installations

In [1]:
!pip install stanza
!pip install gradio
!pip install greek-accentuation==1.2.0
!pip install greek-normalisation

from lib.greek_accentuation.greek_accentuation.syllabify import *
from lib.greek_accentuation.greek_accentuation.characters import *
from lib.greek_accentuation.greek_accentuation.accentuation import *

from greek_normalisation.utils import *
import enum
import gradio as gr
import random
import pandas as pd

---
---
## Generate Exercises

---
### Exercise 1: Find the main verb of the sentence

    Step 0: Get quiz questions

In [ ]:
# define the file name here:
quiz = 'lib/quiz_questions.txt'

# list to hold each line of the file
lines = []
# list of dictionaries for holding the English answer/ Greek answer
exercises = []

# Read in the lines from the file
with open(quiz) as f:
    # create list for holding the exercises
    lines = f.readlines()

# For each line, use regex to grab the answer and full sentence
for sent in lines:
    
    # Get the greek answer
    eng_ans_end = sent.find(':')
    english_answer = sent[0:eng_ans_end]

    greek_answer = sent[eng_ans_end+1:]
    
    # Add everything to our list of dictionaries
    exercises.append({"english answer":english_answer, "greek answer":greek_answer})
    

    Step 1: Parse the sentence

In [15]:
# define the sentence
sentence = "τῷ στρατηγῷ πέμπει τοὺς ἀδελφούς" # for testing -- ultimately, we'll read this in from a file

In [ ]:
import stanza

# stanza.download('grc') 
nlp = stanza.Pipeline('grc') 
doc = nlp(sentence) 

print(doc)
print(doc.entities)

    Step 2: Find the root of the sentence

In [ ]:
root = [word for sent in doc.sentences for word in sent.words if word.deprel=="root"]
print(root)
not_roots = [word for sent in doc.sentences for word in sent.words if word.deprel!="root"]
print(not_roots)

In [ ]:
# get the lemmas for each word
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

    Step 3: Generate other, incorrect answers

In [ ]:
wrong_answers = random.sample(not_roots, 3)
print(wrong_answers)

    Step 4: Put all the potential answers into an array, mix them up

In [ ]:
answers = []
answers.append(root[0].text)

for word in wrong_answers:
    answers.append(word.text)
    
random.shuffle(answers)
    
print(answers)

    Step 5: Get input

In [ ]:
def check(answer):
    if answer == root[0].text:
        return "Correct!"
    return "Not quite"

demo = gr.Interface(description="What is the main verb of the sentence:", fn=check, inputs=[gr.Radio(answers, label=sentence)], outputs="text")

demo.launch()

---
### Exercise 2: Accents (identify the word(s) with the impossible accents)

NOTE: The chosen_words array may vary in length. Ultimately, it would make more sense to have it be a consistent length, even if the number of right/wrong answers changes. 

    Step 0: Place the words into a list

In [ ]:
# define the file name here:
word_file = 'lib/words.txt'

# list to hold each line of the file
all_words = '[]'

# Read in the lines from the file
with open(word_file) as f:
    # create list for holding the exercises
    all_words = f.read().splitlines() 
    
print(all_words)

    Step 1: Randomly choose x number of words from the list. (These are the words which will be in the question)

In [ ]:
# define x here 
x = 4

In [ ]:
# randomly generate x number of words
chosen_words = random.sample(all_words, x)

    Step 2: Randomly split the array into two parts (correct and incorrect answers)

In [ ]:
divider = random.randint(0, x)
correct_words = chosen_words[:divider]
incorrect_words = chosen_words[divider:]

print(incorrect_words)
print(correct_words)

    Step 3: For each 'incorrect' word,
    1. make a list of all the potentially correct accentuations
    2. strip the accents
    3. randomly insert accents
    4. check whether the accents are valid (whether they're in the list)
    5. try to replace the word with a version containing invalid accents

In [ ]:
to_remove = []

# loop through each of the words for which we will assign incorrect accents
for w_index in range(0, len(incorrect_words)):
    
    # get the word, break it into syllables
    w = incorrect_words[w_index]
    s = syllabify(strip_accents(w))
    
    # types of accents
    types = [Accentuation.OXYTONE, Accentuation.PERISPOMENON, Accentuation.PAROXYTONE, Accentuation.PROPERISPOMENON, Accentuation.PROPAROXYTONE]
    
    # make a list of *potentially correct* accentuations for that word
    possible_correct = []
    for a in possible_accentuations(s):
        possible_correct.append(add_accentuation(s, a))

    # make a list of *all* the possible accentuations for that word
    all_accents = []
    for acc_type in types:
        try:
            all_accents.append(add_accentuation(s, acc_type))
        except:
            continue

    # find the difference between the two lists
    possible_incorrect = list(set(all_accents)-set(possible_correct))
    
    # if there are no incorrect answers, remove the word from incorrect_words
    if len(possible_incorrect) == 0:
        to_remove.append(w)
        
    # if there is an incorrect answer, add it to the list
    elif len(possible_incorrect) == 0:
        incorrect_words[w_index] = possible_incorrect[0]
        
    # if there is ore than one incorrect answer, randomly pick one
    else:
        incorrect_words[w_index] = random.choice(possible_incorrect)
    
    print(possible_correct)
    print(all_accents)

for r in to_remove:
    incorrect_words.remove(r)
    
print(incorrect_words)
    



    Step 4: shuffle the array of answers (both correct and incorrect)

In [ ]:
# reinsert the incorrect answers into the original array
chosen_words = correct_words[:] + incorrect_words[:]

# shuffle the array
random.shuffle(chosen_words)

    Step 5: Get input

In [ ]:
def check2(answer):
    if not len(answer) == len(incorrect_words):
        return "You have either selected too many or too few answers"
        
    for i in answer:
        if i not in incorrect_words:
            return "One or more of your answers is incorrect"
        
    return "Correct!"

demo2 = gr.Interface(description="Which of the following words contain impossible accents?:", fn=check2, inputs=[gr.CheckboxGroup(choices=chosen_words, type="value")], outputs="text")

demo2.launch()

---
### Exercise 3: Identifying Syllables
The user is asked to identify the antepenult, penult, or ultima of a given word

    Step 0: Read in the data

In [ ]:
import re
# cognates.tsv contains a list of cognates 
cognates_csv = "lib/cognates.csv"

# convert to dataframes
cognates_df = pd.read_fwf(cognates_csv, header=None, names=["word"])

cognates_list = cognates_df['word'].tolist()

# print(cognates_list)

    Step 1: Choose a greek word from our list randomly

In [ ]:
cur_word = random.choice(cognates_list)

    Step 2: Break the word into syllables (using jtauber's greek-accentuation)

In [ ]:
s = syllabify(cur_word)
# print(s)

    Step 3: Randomly choose either the antepenult, penult, or ultima of the word

In [ ]:
terminology = ['antepenult', 'penult', 'ultima']
cur_terminology = random.choice(terminology)
print(cur_terminology)

    Step 4: Based on this random choice, determine the antepenult, penult, or ultima of the word

In [ ]:
cur_correct_answer = ''

if cur_terminology == 'antepenult':
    cur_correct_answer = antepenult(cur_word)
elif cur_terminology == 'penult':
    cur_correct_answer = penult(cur_word)
else:
    cur_correct_answer = ultima(cur_word)

    Step 5: Ask the user to choose (multiple choice) the antepenult, penult, or ultima of the word (as determined above)

In [ ]:
none_above = "None of the above"
def check3(answer):
    # if the current word does not have an antepenult/penult/ultima
    if cur_correct_answer not in s:
        if answer == none_above:
            return "Correct!"
    else:
        if answer == cur_correct_answer:
            return "Correct!"
    return "Incorrect"

if none_above not in s:
    s.append(none_above)
desc = "What is the " + cur_terminology + " of the word " + cur_word + "?"

demo3 = gr.Interface(description=desc, fn=check3, inputs=[gr.Radio(choices=s, type="value", label='')], outputs="text")

demo3.launch()

---
### Exercise 4: Accents (Rules)
Ask questions abouve which syllables particular accents are allowed to reside within

    Step 0: Read in the data

In [9]:
# cognates.csv contains a list of cognates
cognates_csv = "lib/cognates.csv"

# convert to dataframes
cognates_df = pd.read_fwf(cognates_csv, header=None, names=["word"])

cognates_list = cognates_df['word'].tolist()

# print(cognates_list)

    Step 1: Randomly choose a word, break it into syllables

In [10]:
cur_word = random.choice(cognates_list)
s = syllabify(strip_accents(cur_word))

    Step 2: Randomly choose acute, circumflex, or grave. Identify the penult, ultima, and antepenult of the word & the possible locations of acutes/circumflexes/graves according to the following rules - 
    
    1. circumflex can only be on long penult or ultima
    2. accented penult must have a circumflex if it is long and the ultima is short
    3. otherwise, penult has an acute
    4. an accented ultima CAN have a circumflex if it is long 
    5. an acute on the ultima becomes a grave if another word follows immediately (e.g. if there is not a comma or a full stop)

In [13]:
from lib.greek_accentuation.greek_accentuation.accentuation import *

# randomly choose whether to quiz on acute, circumflex, or grave
accents = ['acute', 'circumflex', 'grave']
cur_accent = random.choice(accents)

# randomly choose the location of the word within a sentence
location = ['another word follows immediately', 'the word is followed by a comma or a full stop']
cur_location = random.choice(location)

# determine the antepenult, penult, and ultima of the word
antepenult = antepenult(cur_word)
penult = penult(cur_word)
ultima = ultima(cur_word)

# identify the possible legal locations of the accent
correct_answers = ['none of the above']

if cur_accent == 'circumflex': # circumflex can be only on long penult or ultima
    if (length(penult) == Length.LONG) and  (length(ultima) == Length.SHORT): # accented penult must have a circumflex if it is long and the ultima is short
        correct_answer = [penult]

    elif length(ultima) == Length.LONG: # if accented ultima is long, it can have a circumflex
        correct_answers = [ultima]
        
elif cur_accent == 'acute':
    if not ((length(penult) == Length.LONG) and  (length(ultima) == Length.SHORT)): #otherwise, penult has an acute
        correct_answers = [penult]
        
elif cur_accent == 'grave':
    if location == 'another word follows immediately':
        correct_answers = [ultima]

print(antepenult)
print(penult)
print(ultima)
print(cur_accent)
print(cur_location)
print(correct_answers)


σο
φί
α
grave
another word follows immediately
['none of the above']


Given a type of accent, have the user determine whether the accent goes on the ultima, penult, or antepenult

In [14]:
def check4(answer):
    # incorrect number of answers were selected
    if not len(answer) == len(correct_answers):
        return "You have either selected too many or too few answers"
        
    for i in answer:
        try:
            # a syllable before the antepenult was selected
            if s.index(i) < s.index(antepenult):
                return "You cannot have an accent before the antepenult"
        except: 
            pass
        # one or more incorrect answers selected
        if i not in correct_answers:
            return "One or more of your answers is incorrect"
        
    return "Correct!"


desc4 = "Is it possible to have a(n) " + cur_accent + " in the word " + strip_accents(cur_word) + "? Assume that " + cur_location + ". If it is not possible, select \'none of the above\'"

if 'none of the above' not in s:
    s.append('none of the above')
    

demo4 = gr.Interface(description=desc4, fn=check4, inputs=[gr.CheckboxGroup(choices=s, type="value", label='')], outputs="text")

demo4.launch()


Running on local URL:  http://127.0.0.1:7865/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7feeb8c82cd0>, 'http://127.0.0.1:7865/', None)

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


---
### Exercise 5: Accents (identify the versions of a word with impossible accents)

    Step 0: Read in the data

In [2]:
# cognates.tsv contains a list of cognates (NOTE: this list is not yet complete)
cognates_csv = "lib/cognates.csv"

# convert to dataframes
cognates_df = pd.read_fwf(cognates_csv, header=None, names=["word"])

cognates_list = cognates_df['word'].tolist()

print(cognates_list)

['ἄθεος', 'ἀθλητής', 'ἀθλητικός', 'βῆτα', 'βιβλιογραφία', 'κατάλυσις', 'καταστροφή', 'δημοκρατία', 'διαχρονικός', 'δρᾶμᾰ', 'ἐκκλησιαστικός', 'εὐλογίζω', 'φαντασία', 'γεωγραφία', 'ἥρως', 'ἱστωρία', 'ὑπροκρίτης', 'ἰδιώτης', 'κρύπτων', 'λιθογραφία', 'μόναρχος', 'νεκρόπολις', 'ὤμεγα', 'ὄμικρον', 'φιλοσοφία', 'πρᾶξις', 'ψυχοανάλυσις', 'ῥαψωδία', 'σοφία', 'θησαυρός', 'ὔψιλον', 'ἔψιλον', 'ξενοφοβία', 'ξυλόφωνος', 'ζηλωτής', 'ζῶογραφία']


    Step 1: Randomly choose a word, find all the possible accentuations

In [3]:
cur_word = random.choice(cognates_list)

s = syllabify(strip_accents(cur_word))

# make a list of correct accentuations for that word
correct = []
for a in possible_accentuations(s):
    correct.append(add_accentuation(s, a))
    
print(cur_word)
print(s)

λιθογραφία
['λι', 'θο', 'γρα', 'φι', 'α']


    Step 2: find impossible accentuations
    
    For each accentuation rule:
     1. break it
     2. add the word to the list of impossible accentuations

In [4]:
from lib.greek_accentuation.greek_accentuation.accentuation import *

# determine the antepenult, penult, and ultima of the word
antepenult = antepenult(cur_word)
penult = penult(cur_word)
ultima = ultima(cur_word)

impossible = []
impossible_feedback = []

cur_word = strip_accents(cur_word)
# Generate incorrect accentuations following these rules:
#--------------------------------
# 1. put an accent before the antepenult (assuming that there is a syllable before the antepenult)
if len(s) > 3:
    impossible.append(cur_word.replace(s[-4], syllable_add_accent(s[-4], Accent.ACUTE)))
    impossible_feedback.append("No accent can go before the antepenult")
    
# 2. if the penult is long and the ultima is short, put an acute on the penult
if (length(penult) == Length.LONG) and (len(ultima) == Length.SHORT):
    impossible.append(cur_word.replace(penult, syllable_add_accent(penult, Accent.ACUTE)))
    impossible_feedback.append("An accented penult must have a circumflex if it is long and the ultima is short")
    
# 3. otherwise, put a circumflex on the penult
else:
    impossible.append(cur_word.replace(penult, syllable_add_accent(penult, Accent.CIRCUMFLEX)))
    impossible_feedback.append("If it is not true that the penult is long and the ultima is short, the penult must have an acute")
    
# 4. put circumflex over short ultimate
if (len(ultima) == Length.SHORT):
    impossible.append(cur_word.replace(ultima, syllable_add_accent(ultima, Accent.CIRCUMFLEX)))
    impossible_feedback.append("an accented ultimate can have a circumflex if it is long")
    
print(impossible)
print(correct)
print(impossible_feedback)


['λιθόγραφια', 'λιθογραφια']
['λιθογραφιά', 'λιθογραφιᾶ', 'λιθογραφία', 'λιθογραφῖα', 'λιθογράφια']
['No accent can go before the antepenult', 'If it is not true that the penult is long and the ultima is short, the penult must have an acute']


    Step 3: Randomly choose several words from each list, combine them in one big list

In [5]:
# randomly generate x number of correct, impossible words
big_list = random.sample(impossible, random.randrange(0, len(impossible))) + random.sample(correct, random.randrange(1, len(correct)))

# shuffle the array
random.shuffle(big_list)

print(big_list)

['λιθόγραφια', 'λιθογράφια']


    Step 4: Gradio interface

In [8]:
def check5(answer):
    if (len(answer) == 0) and (len(correct) != 0):
        return 'Not quite.'
    feedback = []
    for i in answer:
        # one or more incorrect answers selected
        if i in impossible:
            index = impossible.index(str(i))
            feedback.append(impossible_feedback[index] + '\n')
    
    if len(feedback) == 0:
        return 'Correct!'
    return ''.join(feedback)



desc5 = "Identify all of the possible accents"

    

demo5 = gr.Interface(description=desc5, fn=check5, inputs=[gr.CheckboxGroup(choices=big_list, type="value", label='Select all of the correct answers. If none of the answers are correct, just hit \'submit\'')], outputs="text")

demo5.launch()


Running on local URL:  http://127.0.0.1:7864/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7feeb8b85ee0>, 'http://127.0.0.1:7864/', None)

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable
